### **Ingest API Call for Game Data**
Converts API response to a JSON file and save the file to the bronze container

### Configuration

In [30]:
%run /utils/ball_dont_lie_api

In [31]:
%run /azure_utils/sa_functions

### Parameters

In [32]:
season = 2022
team_id = 1

### Call API to Retreive Game Data

In [33]:
current_page = 1
games_dict = get_games(season, team_id, current_page)
total_pages = games_dict['meta']['total_pages']
games_arr = [json.dumps(game) for game in games_dict['data']]


### Metadata For Interacting With ADLS

In [34]:
storage_account_name = 'prizepicksnba'
sas_token = mssparkutils.credentials.getSecret('prizepickskeys', 'sa-sas-token')

### Write File to ADLS

In [35]:
import time

initialize_storage_account_sas(storage_account_name, sas_token)
create_directory('bronze', f'games/season={season}/team_id={team_id}')
upload_single_line_json_file_to_directory('bronze', f'games/season={season}/team_id={team_id}', f'game_file{current_page}.json', games_arr)

current_page += 1

while current_page <= total_pages:
    games_dict = get_games(season, team_id, current_page)
  
    games_arr = [json.dumps(game) for game in games_dict['data']]

    initialize_storage_account_sas(storage_account_name, sas_token)
    create_directory('bronze', f'games/season={season}/team_id={team_id}')
    upload_single_line_json_file_to_directory('bronze', f'games/season={season}/team_id={team_id}', f'game_file{current_page}.json', games_arr)

    current_page += 1

    # Delay to avoid rate limit of API
    time.sleep(1)